In [0]:
import numpy as np

In [0]:
"""
LUT = {-1:tbd, 0~8: #mines near by, -9: mines}
Finished
"""

def generate_board(board_size, num_mines, num_hints):
    board = np.full((board_size, board_size), -1, dtype=int)

    idx = [(x, y) for x in range(board_size) for y in range(board_size)]
    # Random select where mines and hints placed
    idx_choice = np.random.choice(len(idx), num_mines + num_hints,
                                  replace=False)
    choice_map = [idx[n] for n in idx_choice]
    mines_map = choice_map[:num_mines]
    hints_map = choice_map[num_mines:]

    board_with_mines = find_hints(board, mines_map, hints_map)
    return board, board_with_mines

def find_hints(board, mines_map, hints_map):
    mines_set = set(mines_map)
    for idx in hints_map:
        # Find which place to be count
        count_place = possible(idx, len(board))
        cnt = 0
        for place in count_place:
            if place in mines_set:
                cnt += 1
        # Update board with relative hint
        board[idx[0]][idx[1]] = cnt
    # Generate board with mines
    board_with_mines = board.copy()
    for idx in mines_map:
        board_with_mines[idx[0]][idx[1]] = -9
    return board_with_mines

def possible(idx, board_size):
    ways = [(x, y) for x in range(-1, 2) for y in range(-1, 2) if
            (x, y) != (0, 0)]
    surrounding = [(idx[0] + way[0], idx[1] + way[1]) for way in ways]
    return [s for s in surrounding if is_available(s, board_size)]

def is_available(idx, board_size):
    return 0 <= idx[0] < board_size and 0 <= idx[1] < board_size

In [0]:
import queue

In [0]:
class Minesweeper:
    def __init__(self, board_size, num_mines, board):
        self.board_size = board_size
        self.num_mines = num_mines
        self.board = board
        self.v_list, self.h_list = self.make_list()
        self.v_set, self.h_set = set(self.v_list), set(self.h_list)
        self.v_surrounds = self.make_v_surrounds()
        self.h_surrounds = self.make_h_surrounds()
    
    def make_list(self):
        v_list = list()
        h_list = list()
        for i in range(self.board_size):
            for j in range(self.board_size):
                idx = (i, j)
                if board[i][j] == -1:
                    v_list.append(idx)
                else:
                    h_list.append(idx)

        return v_list, h_list

    # Main function.
    def find_mines(self, mode):
        """Generate init stack."""
        q = queue.LifoQueue()
        domains = {variable: [0, 1] for variable in self.v_list}
        domains = self.update_domains(list(), domains)

        node = (list(), domains.copy())
        q.put(node)
        while not q.empty():
            node = q.get()
            assigned, domains = node
            # Correct.
            if len(assigned) == len(self.v_list):
                if self.correct(assigned):
                    return assigned 
            
            # There is variable which has no domain.
            if len(assigned) + len(domains) != len(self.v_list):
                continue

            for variable, domain in domains.items():
                for mine in domain:
                    new_assigned = assigned.copy()
                    new_assigned.append((variable, mine))
                    new_domains = domains.copy()
                    new_domains.pop(variable)
                    new_domains = self.update_domains(new_assigned, new_domains)
                    node = (new_assigned, new_domains)
                    q.put(node)

    def correct(self, assigned):
        cnt = 0
        for node in assigned:
            variable, mine = node
            cnt += mine
        if cnt != self.num_mines:
            return False
        
        assigned_dict = dict(assigned)
        for hint in self.h_list:
            cnt = 0
            for variable in self.h_surrounds[hint]:
                cnt += assigned_dict[variable]
            i, j = hint
            if cnt != self.board[i][j]:
                return False
        
        return True

    def update_domains(self, assigned, old_domains):
        domains = old_domains.copy()

        assigned_dict = dict(assigned)
        for hint in self.h_list:
            cnt = 0
            for variable in self.h_surrounds[hint]:
                if variable in assigned_dict:
                    cnt += assigned_dict[variable]
            
            i, j = hint
            for variable in self.h_surrounds[hint]:
                if variable in domains:
                    if self.board[i][j] < cnt:
                        domains.pop(variable)
                    elif self.board[i][j] == cnt:
                        domains[variable] = [0]
                    elif self.board[i][j] > cnt and domains[variable] != [0]:
                        domains[variable] = [9, 1]
        
        return domains

    # Make a dict {hints: [surround variables]}
    def make_h_surrounds(self):
        h_surrounds = dict()
        for hint in self.h_list:
            surrounds = self.find_surrounds(hint)
            surrounds = [place for place in surrounds if place in self.v_set]
            h_surrounds.update({hint: surrounds})
        return h_surrounds
    
    def make_v_surrounds(self):
        v_surrounds = dict()
        for variable in self.v_list:
            surrounds = self.find_surrounds(variable)
            surrounds = [place for place in surrounds if place in self.h_set]
            v_surrounds.update({variable: surrounds})
        return v_surrounds
    
    def find_surrounds(self, idx):
        surrounds = list()
        ways = [(-1, -1), (-1, 0), (-1, 1),
                ( 0, -1),          ( 0, 1),
                ( 1, -1), ( 1, 0), ( 1, 1)]
        i, j = idx
        for way in ways:
            x, y = way
            if 0 <= i+x < self.board_size and 0 <= j+y < self.board_size:
                surrounds.append((i+x, j+y))

        return surrounds

In [0]:
board_size = 6
num_mines = 10
num_hints = 20

In [0]:
board, board_with_mines = generate_board(board_size, num_mines, num_hints)

In [0]:
m = Minesweeper(board_size, num_mines, board)
assigned = m.find_mines(0)

In [8]:
board_find = board.copy()
for node in assigned:
    variable, mine = node
    i, j = variable
    if mine == 1:
        board_find[i][j] = -9

mines_set = set()
for i in range(board_size):
    for j in range(board_size):
        if board_with_mines[i][j] == -9:
            mines_set.add((i, j))

mines_set_find = set()
for i in range(board_size):
    for j in range(board_size):
        if board_find[i][j] == -9:
            mines_set_find.add((i, j))

print(mines_set - mines_set_find)
print(mines_set_find - mines_set)

set()
set()


In [9]:
print(board)
print(board_with_mines)
print(board_find)

[[ 1  1 -1  2 -1 -1]
 [ 1 -1  2  3 -1 -1]
 [ 1  2 -1 -1 -1 -1]
 [ 1  2 -1 -1  1  1]
 [-1  2  2  1 -1  0]
 [ 2 -1  2 -1  1  0]]
[[ 1  1 -1  2 -9 -9]
 [ 1 -9  2  3 -9 -9]
 [ 1  2 -9 -1 -1 -9]
 [ 1  2 -1 -1  1  1]
 [-9  2  2  1 -1  0]
 [ 2 -9  2 -9  1  0]]
[[ 1  1 -1  2 -9 -9]
 [ 1 -9  2  3 -9 -9]
 [ 1  2 -9 -1 -1 -9]
 [ 1  2 -1 -1  1  1]
 [-9  2  2  1 -1  0]
 [ 2 -9  2 -9  1  0]]
